In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n = 4
dim = 2**n
qubit_wires = range(n)
layers = 4

params = np.load('Potential_4qubits_0.01serr_params_9seed_Adam.npy')

In [ ]:
def create_A(gate_set, coefficient_set):
    A = np.zeros((dim,dim), dtype = complex)
    for i in range(len(gate_set)):
        An = np.zeros((2,2))
        for j in range(n):
            if j == 0:
                if gate_set[i][j] == 'I': 
                    An = np.eye(2)
                elif gate_set[i][j] == 'X':
                    An = qml.PauliX.compute_matrix()
                elif gate_set[i][j] == 'Y':
                    An = qml.PauliY.compute_matrix()
                elif gate_set[i][j] == 'Z':
                    An = qml.PauliZ.compute_matrix()
            else:
                if gate_set[i][j] == 'I': 
                    An = np.kron(An, np.eye(2))
                elif gate_set[i][j] == 'X':
                    An = np.kron(An, qml.PauliX.compute_matrix())
                elif gate_set[i][j] == 'Y':
                    An = np.kron(An, qml.PauliY.compute_matrix())
                elif gate_set[i][j] == 'Z':
                    An = np.kron(An, qml.PauliZ.compute_matrix())

        A = np.add(A, coefficient_set[i]*An)
    return A

In [ ]:
gate_set, coefficient_set =  [], []

fname = f'{4}x{4}sites_potential.txt'
with open(fname, 'r') as f:
    for line in f.readlines():
        gate, real, imag = line.split()
        gate_set.append(gate)
        coefficient_set.append(complex(float(real), float(imag)))
        
A1 = create_A(gate_set, coefficient_set)
b1 = np.load(f'{4}x{4}_bnorm.npy')
b1  = b1/np.linalg.norm(b1)

In [ ]:
#single layer
def layer(qubit_wires, parameters):
    n = len(qubit_wires)
    if n%2==0:
        qml.broadcast(qml.CNOT,wires=qubit_wires, pattern='double')
        qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
        qml.broadcast(qml.CNOT,wires=qubit_wires[1:], pattern='double')
        qml.broadcast(qml.RY, wires=qubit_wires[1:-1], parameters=parameters[n:2*(n-1)], pattern='single')
    else:
        qml.broadcast(qml.CNOT,wires=qubit_wires, pattern='double')
        qml.CNOT(wires=[qubit_wires[0], qubit_wires[-1]])
        qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
        qml.broadcast(qml.CNOT,wires=qubit_wires[1:], pattern='double')
        qml.CNOT(wires=[qubit_wires[0], qubit_wires[-1]])
        qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[n:2*n], pattern='single')

#Fixed anstaz
def apply_fixed_ansatz(qubit_wires, parameters, layers):  
    n = len(qubit_wires)  
    qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
    
    if n%2==0:
        for l in range(layers):
            layer(qubit_wires, parameters[n+2*(n-1)*l:n+2*(n-1)*l+2*(n-1)])
    else:
        for l in range(layers):
            layer(qubit_wires, parameters[n+2*n*l:n+2*n*l+2*n])

In [ ]:
dev = qml.device('lightning.qubit', wires=qubit_wires)

@qml.qnode(dev)
def circuit_fielity(parameters):
    apply_fixed_ansatz(qubit_wires, parameters, layers)
    return qml.state()

def calc_fidelity(state_vec, A, b):
    Ainv = np.linalg.inv(A)
    Ainvprodb = np.matmul(Ainv,b)
    Ainvprodb_norm = Ainvprodb/np.linalg.norm(Ainvprodb)  
    fidelity = np.dot(np.array(Ainvprodb_norm), state_vec)
    return fidelity

In [ ]:
import numpy as np

def setup_system(n):
    # Number of interior points in each direction (n x n grid)
    N = n * n  # Interior points

    # Initialize the matrix A and the vector b
    A = np.zeros((N, N))
    b = np.zeros(N)

    # Boundary conditions
    phi_top = 0.25
    phi_bottom = 0
    phi_left = 0
    phi_right = 0

    # Fill the matrix A and vector b based on the discretized Laplace equation
    for i in range(n):
        for j in range(n):
            # Current index in the unknowns vector
            k = i * n + j

            # Fill in the matrix A
            A[k, k] = -4  # Main diagonal element

            if i > 0:
                A[k, k - n] = 1  # Above element
            else:
                    b[k] -= phi_top  # Top boundary condition
            if i < n - 1:
                A[k, k + n] = 1  # Below element
            else:
                b[k] -= phi_bottom  # Bottom boundary condition

            if j > 0:
                A[k, k - 1] = 1  # Left element
            else:
                b[k] -= phi_left  # Left boundary condition

            if j < n - 1:
                A[k, k + 1] = 1  # Right element
            else:
                b[k] -= phi_right  # Right boundary condition

    return A, b

def solve_system(A, b):
    # Solve the system A * x = b
    x = np.linalg.solve(A, b)
    return x

# Set up a 4x4 grid of interior points, which means n = 4
n = 4
A2, b2 = setup_system(n)
potentials = solve_system(A2, b2)

# Reshape the solution to the grid
potential_grid = potentials.reshape((n, n))

In [ ]:
potentials = solve_system(A1.real, b1.real)
potential_grid = potentials.reshape((4, 4))/np.linalg.norm(potentials)

In [ ]:
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.sans-serif": "Palatino",})

plt.figure(figsize=(5,5))

plt.imshow(potential_grid, cmap='hot', norm='linear')
plt.xticks([])
plt.yticks([])
plt.xlabel(r'$V=0$', fontsize=10)
plt.ylabel(r'$V=0$', fontsize=10)
plt.text(1.02, 0.5, r'$V=0$', fontsize=10, rotation=270, va='center', ha='left', transform=plt.gca().transAxes)
plt.title(r'$V=V_0$', fontsize=10)
# plt.colorbar(anchor=(0.5,0.5), shrink=0.5)
plt.colorbar(location='bottom')
plt.savefig('4x4_heatmap_analytic.pdf', bbox_inches='tight')

In [ ]:
sol = circuit_fielity(params)

In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.sans-serif": "Palatino",})

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 2.75))

sol_graph = (sol- np.min(sol))/np.max(sol- np.min(sol))
pot_grid_graph = (potential_grid- np.min(sol))/np.max(sol- np.min(sol))

im1 = ax1.imshow(sol.real, cmap='hot')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlabel(r'$V=0$', fontsize=10)
ax1.set_ylabel(r'$V=0$', fontsize=10)
plt.text(-0.65, 0.5, r'$V=0$', fontsize=10, rotation=270, va='center', ha='left', transform=plt.gca().transAxes)
plt.text(1.02, 0.5, r'$V=0$', fontsize=10, rotation=270, va='center', ha='left', transform=plt.gca().transAxes)
ax1.set_title(r'$V=V_0$', fontsize=10)
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes(position ="bottom", size="5%", pad=0.25)
cbar1 = plt.colorbar(im1, cax=cax1, orientation="horizontal", ticks = [np.min(sol) , np.max(sol)], format = "%.2f")

im2 = ax2.imshow(-1*potential_grid, cmap='hot', vmin= np.min(sol), vmax=np.max(sol))
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$V=0$', fontsize=10)
ax2.set_ylabel(r'$V=0$', fontsize=10)
ax2.set_title(r'$V=V_0$', fontsize=10)
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("bottom", size="5%", pad=0.25)
cbar2 = plt.colorbar(im2, cax=cax2, orientation="horizontal", ticks = [np.min(sol) , np.max(sol)], format = "%.2f")

plt.savefig('potential_grids_4x4.pdf', bbox_inches='tight')